# Calling C++

**Table of contents**<a id='toc0_'></a>    
- 1. [Imports](#toc1_)    
- 2. [Setup](#toc2_)    
- 3. [Visual Studio compiler](#toc3_)    
  - 3.1. [Link to C++](#toc3_1_)    
  - 3.2. [Calling functions](#toc3_2_)    
  - 3.3. [Re-compile](#toc3_3_)    
  - 3.4. [See all availible compiler options](#toc3_4_)    
- 4. [Intel compiler](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

The C++ file used in this example is in **cppfuncs/example.cpp**. 

**Read the following information:**

1. **Functions are automatically detected**. The provided .cpp file should include:

    `#define EXPORT extern "C" __declspec(dllexport)`

    A function to be called in Python should be **decorated** as e.g.:
 
    `EXPORT void fun_nostruct(double *X, double *Y, int N, double a, double b, int threads, char *txt)`
 
    **Extra:** Functions are also detected in *directly* included .cpp files.
 
2. The following **argument types** are allowed: `int` `int*` `double` `double*` `bool` `bool*` and `char*` (for strings) + **user-defined structs** (see below).
3. The foolowing **return types** are allowed: `void` `int` `double` and `bool`.
4. **Class:** `cppfile = cpptools.link_to_cpp(filename,force_compile,structsmap,options,use_log_print,do_print)` returns an interface to an object with the **C++ functions as methods**.
 
    `filename (str)`: C++ file with .cpp extension (full path)
    
    `force_compile (bool,optional)`: compile even if .dll is present (default is True)
    
    `structsmap (dict,optional)`: struct names as keys and associated pythonobj used in C++ as values (see below)
    
    `options (dict,optional)`: compiler options (see below)
    
    `use_log (bool,optional)`: assumes log is printed to filename.log (deleted afterwards, saved in self.log[funcname])
    
    `print_log (bool,optional)`: print log to screen when function is called
    
    `do_print (bool,optional)`: print progress
    
5. **User-defined structs:** Assume `par` is a Python dict-like object (e.g. SimpleNamespace) then `structsmap={'par_struct':par}` allows `par_struct*` to be used as a type in C++ after `#include "par_struct.cpp"`, which is automatically written (in the same folder as the *filename.cpp* file) with automatically inferred fields and their types.
 
6. **Compiler options:** Dictionary with the following elements:

    `compiler (str)`: compiler choice (*vs* (default) or *intel*)
    
    `vs_path (str)`: path to vs compiler (if *None* (default) then newest version found is used, e.g. *C:/Program Files/Microsoft Visual Studio/2022/Community/VC/Auxiliary/Build/*)
    
    `intel_path (str)`: path to intel compiler
    
    `flags (str)`: flags (vs default: '/LD /EHsc /Ox /openmp', intel default: '/LD /EHsc /O3 /openmp')
    
    `additional_cpp (str)`: additional cpp files to include (default is '')
    
    `dllfilename (str)`: filename of resulting dll file (if *None* (default) it is *filename.dll* where *filename.cpp*)
    
    `macros (dict/list)`: preprocessor macros (default is *None*)
        
7. **Methods:**

    `cppfile.delink()`: Delink C++ file (release the .dll file, so that it can be overwritten when e.g. re-compiling).
    
    `cppfile.recompile()`: Delink, re-compile and re-link to C++ file.
    
    `cppfile.clean_up()`: Delink C++ file, remove .dll file, and remove any struct files.

## 1. <a id='toc1_'></a>[Imports](#toc0_)

In [1]:
%load_ext autoreload
%autoreload 2

import os
from types import SimpleNamespace
import numpy as np
import numba as nb

from EconModel import cpptools

## 2. <a id='toc2_'></a>[Setup](#toc0_)

In [2]:
par = SimpleNamespace()
par.N = 10
par.X = np.linspace(0,10,par.N)
par.Z = np.ones(par.N,dtype=int)
par.a = 2.0
par.b = 1.0
par.threads = 4
par.txt = 'a'
par.txtlist = 'N|threads'

sol = SimpleNamespace()
sol.Y = np.zeros(par.N)

## 3. <a id='toc3_'></a>[Visual Studio compiler](#toc0_)

### 3.1. <a id='toc3_1_'></a>[Link to C++](#toc0_)

In [3]:
example = cpptools.link_to_cpp('cppfuncs/example.cpp',structsmap={'par_struct':par,'sol_struct':sol},do_print=True)

Linking to: cppfuncs/example.cpp

### finding all included files ###

logs.cpp
par_struct.cpp
sol_struct.cpp
example_include.cpp

### writing structs ###

cppfuncs/par_struct.cpp
 int N;
 double* X;
 int* Z;
 double a;
 double b;
 int threads;
 char* txt;
 char* txtlist;

cppfuncs/sol_struct.cpp
 double* Y;

### analyzing cppfuncs/example.cpp ###

function: fun
return type: void
argument types: ['par_struct*', 'sol_struct*']

function: fun_nostruct
return type: void
argument types: ['double*', 'double*', 'int*', 'int', 'double', 'double', 'int', 'char*']

### analyzing cppfuncs/logs.cpp ###

### analyzing cppfuncs/par_struct.cpp ###

### analyzing cppfuncs/sol_struct.cpp ###

### analyzing cppfuncs/example_include.cpp ###

function: fun_nostruct_alt
return type: void
argument types: ['double*', 'double*', 'int*', 'int', 'double', 'double', 'int', 'char*']

function: I_DO_NOTHING
return type: double
argument types: []

### compiling and linking ###

compile.bat:
cd /d "C:/Program Files/

terminal:

(c:\Users\gmf123.UNICPH\AppData\Local\anaconda3) c:\Users\gmf123.UNICPH\Documents\repositories\EconModelNotebooks>cd /d "C:/Program Files/Microsoft Visual Studio/2022/Community/VC/Auxiliary/Build/" 

(c:\Users\gmf123.UNICPH\AppData\Local\anaconda3) C:\Program Files\Microsoft Visual Studio\2022\Community\VC\Auxiliary\Build>call vcvarsall.bat x64 
**********************************************************************
** Visual Studio 2022 Developer Command Prompt v17.0.6
** Copyright (c) 2021 Microsoft Corporation
**********************************************************************
[vcvarsall.bat] Environment initialized for: 'x64'
example.cpp
setup_omp.cpp
Generating Code...
Microsoft (R) Incremental Linker Version 14.30.30709.0
Copyright (C) Microsoft Corporation.  All rights reserved.

/out:example.dll 
/dll 
/implib:example.lib 
example.obj 
setup_omp.obj 
   Creating library example.lib and object example.exp

Microsoft (R) C/C++ Optimizing Compiler Version 19.30.30709 

### 3.2. <a id='toc3_2_'></a>[Calling functions](#toc0_)

**Note:** The C++ file writes to `example.log`. The content is printed here, and the file is then deleted.

In [4]:
example.fun(par,sol)


fun(...)
omp_get_thread_num() =  0, omp_get_num_procs() = 72
omp_get_thread_num() =  1, omp_get_num_procs() = 72
omp_get_thread_num() =  2, omp_get_num_procs() = 72
string-value: a
looked up value par->a = 2
is "a" in "N|threads": false
looked up value par->N = 10
looked up value par->threads = 4



In [5]:
example.fun(par,sol)
assert np.allclose(par.X*(par.a+par.b)*par.Z,sol.Y) 


fun(...)
omp_get_thread_num() =  0, omp_get_num_procs() = 72
omp_get_thread_num() =  1, omp_get_num_procs() = 72
omp_get_thread_num() =  2, omp_get_num_procs() = 72
string-value: a
looked up value par->a = 2
is "a" in "N|threads": false
looked up value par->N = 10
looked up value par->threads = 4



In [6]:
par.Y = np.zeros(par.N)
example.fun_nostruct(par.X,sol.Y,par.Z,par.N,par.a,par.b,par.threads,par.txt)
assert np.allclose(par.X*(par.a+par.b),sol.Y)


fun_nostruct(...)
omp_get_thread_num() =  0, omp_get_num_procs() = 72
omp_get_thread_num() =  3, omp_get_num_procs() = 72
string-value: a
test was succesfull



In [7]:
sol.Y = np.zeros(par.N)
example.fun_nostruct_alt(par.X,sol.Y,par.Z,par.N,par.a,par.b,par.threads,par.txt)
assert np.allclose(par.X*(par.a+par.b),sol.Y)

omp_get_thread_num() =  0, omp_get_num_procs() = 72


omp_get_thread_num() =  1, omp_get_num_procs() = 72
omp_get_thread_num() =  3, omp_get_num_procs() = 72


### 3.3. <a id='toc3_3_'></a>[Re-compile](#toc0_)

In [8]:
example.recompile(print_log=False)

In [9]:
res = example.I_DO_NOTHING()
assert np.isclose(res,5.0)

**Manual print of log:**

In [10]:
print(example.log['I_DO_NOTHING'])


I do nothing!



**Clean up:**

In [11]:
example.clean_up()

### 3.4. <a id='toc3_4_'></a>[See all availible compiler options](#toc0_)

In [12]:
example.options

{'nlopt_lib': 'cppfuncs/nlopt-2.4.2-dll64/libnlopt-0.lib',
 'tasmanian_lib': 'cppfuncs/TASMANIAN-7.0/lib/tasmaniansparsegrid.lib',
 'compiler': 'vs',
 'vs_path': 'C:/Program Files/Microsoft Visual Studio/2022/Community/VC/Auxiliary/Build/',
 'intel_path': 'C:\\Program Files (x86)\\Intel\\oneAPI',
 'flags': None,
 'additional_cpp': '',
 'macros': None,
 'dllfilename': None}

The **default flags** (when value is `None`) are:

* **vs**: /LD /EHsc /Ox /openmp
* **intel**: /LD /EHsc /O3 /openmp              

## 4. <a id='toc4_'></a>[Intel compiler](#toc0_)

In [13]:
example = cpptools.link_to_cpp('cppfuncs/example.cpp',
                               structsmap={'par_struct':par,'sol_struct':sol},
                               options={'compiler':'intel'})

example.fun(par,sol)
assert np.allclose(par.X*(par.a+par.b),sol.Y)

example.clean_up()


fun(...)
omp_get_thread_num() =  2, omp_get_num_procs() = 72
string-value: a
looked up value par->a = 2
is "a" in "N|threads": false
looked up value par->N = 10
looked up value par->threads = 4

